# Проект для «Викишоп»

#### Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 
Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.
Постройте модель со значением метрики качества F1 не меньше 0.75. 

## Подготовка

### Ипорт библиотек

In [1]:
#pip install pymystem3
#!pip install --upgrade pip
!python -m textblob.download_corpora

/opt/conda/bin/python: Error while finding module specification for 'textblob.download_corpora' (ModuleNotFoundError: No module named 'textblob')


In [2]:
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas()
import re
import time
from ipywidgets import FloatProgress
from sklearn.model_selection import train_test_split, GridSearchCV

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords as nltk_stopwords
stopwords = set(nltk_stopwords.words('english'))

from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
from lightgbm import LGBMClassifier

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
toxic_comments=pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv',sep=',')

In [4]:
def dataset_info(dataset,name):
    display(f'Анализ датафрейма {name}')
    display(dataset.head(5))
    display(f'Количество пустых ячеек:')
    display(dataset.isnull().sum())
    display(dataset.info())
    display(f'Количество явных дубликатов в датасете:{dataset.duplicated().sum()}' )
    dataset=dataset.drop_duplicates()
    display(f'Количество явных дубликатов в датасете после удаления:{dataset.duplicated().sum()}' )

In [5]:
dataset_info(toxic_comments, 'toxic_comments')

'Анализ датафрейма toxic_comments'

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


'Количество пустых ячеек:'

Unnamed: 0    0
text          0
toxic         0
dtype: int64

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


None

'Количество явных дубликатов в датасете:0'

'Количество явных дубликатов в датасете после удаления:0'

###  Выводы
Текст необходимо лемматизировать и убрать символы. Пропуски и дубликаты отсутствуют

Расчетное время лемматизации у меня было в районе 30 часов. Может что-то не так делал, но по образу и подобию из лекций взял материал. На хабре нашел метод, который переработал немного и применил в ячейке ниже. Лемматизация не сработала почему-то(  https://habr.com/ru/articles/503420/ Нашел еще функцию и быстрее ее ничего не смог сделать. Взял за основу в работе 

In [6]:
toxic_comments=toxic_comments.drop('Unnamed: 0',axis=1)
toxic_comments['text'] = toxic_comments['text'].str.lower()

def clear_text(text):
    text = re.sub(r'[^a-z]', ' ', text)
    text = text.split()
    return " ".join(text)
toxic_comments['text_edited'] = toxic_comments['text'].progress_apply(lambda x: clear_text(x))

  0%|          | 0/159292 [00:00<?, ?it/s]

In [7]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [8]:


wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    """ Converts Penn Treebank tags to WordNet. """
    morphy_tag = {'NN':'n', 'JJ':'a',
                  'VB':'v', 'RB':'r'}
    try:
        return morphy_tag[penntag[:2]]
    except:
        return 'n' 

def lemmatize_sent(text): 
    # Text input is string, returns lowercased strings.
    return [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) 
            for word, tag in pos_tag(word_tokenize(text))]


toxic_comments['lemm'] =toxic_comments['text_edited'].progress_apply(lemmatize_sent)
toxic_comments['lemm']=toxic_comments['lemm'].astype(str)

  0%|          | 0/159292 [00:00<?, ?it/s]

# Обучение модели

In [9]:
RANDOM_STATE=42
TEST_SIZE=0.2

In [10]:
y = toxic_comments['toxic']
X = toxic_comments['lemm']

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=TEST_SIZE,random_state=RANDOM_STATE)

count_tf_idf = TfidfVectorizer(stop_words=list(stopwords))
X_train = count_tf_idf.fit_transform(X_train)
X_test = count_tf_idf.transform(X_test)


In [11]:
start_time = time.time()

model_LR = LogisticRegression(random_state=RANDOM_STATE)
params = [{'solver':['newton-cg', 'lbfgs', 'liblinear'],
                'C':[0.1, 1, 10]}]
clf = GridSearchCV(model_LR, params, scoring='f1',cv=2)
clf.fit(X_train, y_train)
LR_best_params = clf.best_params_
model_LR.set_params(**LR_best_params)
model_LR.fit(X_train, y_train)

end_time = time.time()

education_time_LR = end_time - start_time
f1_LR = clf.best_score_

In [12]:
start_time = time.time()
param_search_lgb = {'max_depth': [4, 6, 10, 12],
                    'num_leaves': [30, 40, 50, 60]}

model_LGBMC = LGBMClassifier(n_estimators=100,learning_rate=0.3,class_weight='balanced',random_state=RANDOM_STATE,force_col_wise=True)

clf = GridSearchCV(model_LGBMC, param_grid=param_search_lgb, scoring='f1',cv=2,n_jobs=-1)
clf.fit(X_train, y_train)
LGBMC_best_params = clf.best_params_
model_LGBMC.set_params(**LGBMC_best_params)
model_LGBMC.fit(X_train, y_train)

end_time = time.time()

education_time_LGBMC = end_time - start_time
f1_LGBMC = clf.best_score_

In [13]:
start_time = time.time()
param_search_SGDC =  {'alpha': [0.0001, 0.001, 0.01, 0.1]}

model_SGDC = SGDClassifier(max_iter=100,tol=0.001,class_weight='balanced',random_state=RANDOM_STATE)

clf = GridSearchCV(model_SGDC, param_grid=param_search_SGDC, scoring='f1',cv=2,n_jobs=-1)
clf.fit(X_train, y_train)
SGDC_best_params = clf.best_params_
model_SGDC.set_params(**SGDC_best_params)
model_SGDC.fit(X_train, y_train)

end_time = time.time()

education_time_SGDC = end_time - start_time
f1_SGDC = clf.best_score_

###  Выводы
Обучены разные модели: LogisticRegression,LGBMClassifier,SGDClassifier

## Анализ моделей

In [14]:
index = ['LogisticRegression',
         'LGBMClassifier',
        'SGDClassifier']
data = {'F1':[f1_LR,f1_LGBMC,f1_SGDC],'education_time':[education_time_LR,education_time_LGBMC,education_time_SGDC]}
display(pd.DataFrame(data=data, index=index))

,F1,education_time
LogisticRegression,0.750696,399.720850
LGBMClassifier,0.744974,1987.077929
SGDClassifier,0.730765,4.542535


In [15]:
start_time = time.time()
target_predict = model_LR.predict(X_test)
end_time = time.time()
prediction_time_LR_test = end_time - start_time

f1_LR_test = f1_score( y_test,target_predict)

display('prediction_time_LogisticRegression_test ',prediction_time_LR_test)
display('f1_LogisticRegression_test ',f1_LR_test)

'prediction_time_LogisticRegression_test '

0.0035238265991210938

'f1_LogisticRegression_test '

0.7731741573033708

###  Выводы
По результатам анализа моделей можно сделать вывод, что величина f1 самая высокая (0,75) у логистической регрессии. 
Стоит остановить выбор на данной модели. f1 0,77